In [190]:
%load_ext autoreload
%autoreload 2

from tabula import read_pdf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [377]:
pdf_name = '../sample-data/B1705_2000606271.pdf'

In [378]:
json_obj = read_pdf(pdf_name, output_format='json', guess=False)
# top left bottom right

In [379]:
json_obj

[{'extraction_method': 'stream',
  'top': 0.0,
  'left': 0.0,
  'width': 573.3900146484375,
  'height': 818.280029296875,
  'data': [[{'top': 0.0, 'left': 0.0, 'width': 0.0, 'height': 0.0, 'text': ''},
    {'top': 0.0, 'left': 0.0, 'width': 0.0, 'height': 0.0, 'text': ''},
    {'top': 26.59,
     'left': 321.0,
     'width': 197.8400115966797,
     'height': 4.71999979019165,
     'text': 'lab-adelaide@idexx.com lab-sydney@idexx.com'}],
   [{'top': 0.0, 'left': 0.0, 'width': 0.0, 'height': 0.0, 'text': ''},
    {'top': 0.0, 'left': 0.0, 'width': 0.0, 'height': 0.0, 'text': ''},
    {'top': 39.59,
     'left': 321.0,
     'width': 211.00999450683594,
     'height': 4.71999979019165,
     'text': 'lab-brisbane@idexx.com lab-melbourne@idexx.com'}],
   [{'top': 0.0, 'left': 0.0, 'width': 0.0, 'height': 0.0, 'text': ''},
    {'top': 0.0, 'left': 0.0, 'width': 0.0, 'height': 0.0, 'text': ''},
    {'top': 52.59,
     'left': 321.0,
     'width': 70.27000427246094,
     'height': 4.71999979019

In [380]:
def get_obj_with_text(json_obj, text):
    for i in json_obj:
        objs = i['data']
        for obj in objs:
            for o in obj:
                if o['text'] == text:
                    return o
                
def get_obj_with_text_contains(json_obj, text):
    for i in json_obj:
        objs = i['data']
        for obj in objs:
            for o in obj:
                if text in o['text']:
                    return o

In [381]:
table_start = 0
o = get_obj_with_text(json_obj, 'Range')
table_start = o['top'] + o['height']
table_start

199.30999979019165

In [382]:
o = get_obj_with_text_contains(json_obj, 'BLOOD SMEAR')
blood_smear_top = o['top']


In [383]:
for i in json_obj:
    objs = i['data']
    for j, _ in enumerate(objs):
        if j < len(i['data']) - 1:
            prev = objs[j][0]
            after = objs[j+1][0]
            
            diff = get_obj_vals(after['top'] - prev['top'])
            if diff < 25 and diff > 20 and after['top'] > table_start:
                print(f"Found space between {after['text']} and {prev['text']} of {diff}")
                table_end = after['top']
                break
            


Found space between This report has been automatically generated based on accepted physiological limits for the species and criteria and platelet clumps of 23.309999999999945


In [384]:
df = read_pdf(
    pdf_name, output_format='dataframe', 
    guess=False, area=(table_start, 0, table_end, 1000), 
    pandas_options={'header': None})

In [385]:
df = df.loc[:, 0:1]

In [386]:
df

,0,1
0,RBC,11.2 x10^12/L
1,HAEMOGLOBIN,131 g/L
2,HAEMATOCRIT,0.40 L/L
3,MCV,36 fL
4,MCH,12 pg
5,MCHC,328 g/L
6,PLATELETS,Clumped and adequate
7,WBC,6.6 x10^9/L
8,NEUTROPHILS%,25 %
9,NEUTROPHILS,1.7 x10^9/L


In [387]:
def parse_value(val):
    try:
        return float(val)
    except ValueError:
        text = val.split(' ')
        try:
            return float(text[0])
        except ValueError:
            return val


In [388]:
df[1] = df[1].apply(parse_value)

In [389]:
df

,0,1
0,RBC,11.2
1,HAEMOGLOBIN,131
2,HAEMATOCRIT,0.4
3,MCV,36
4,MCH,12
5,MCHC,328
6,PLATELETS,Clumped and adequate
7,WBC,6.6
8,NEUTROPHILS%,25
9,NEUTROPHILS,1.7
